In [1]:
from numpy import *
from scipy.sparse import *
from sklearn.preprocessing import *

In [4]:
#trackDuration=genfromtxt("tracks_final.csv",usecols=[0,2],skip_header=1,dtype=[("trackID",int32),("duration",int32)])
#trackDuration.sort(order="trackID")
#trackDuration=array(list(map(lambda x: [x[0],x[1]],trackDuration)))
#save("fucking_ordered_track_duration.npy",trackDuration)
trackDuration=load("fucking_ordered_track_duration.npy")
trackDuration.shape
durations=trackDuration[:,1]#contiene le duration delle 100k track

In [51]:
mymask=where(trackDuration[:,1]==-1)[0]
A=array(ones(100000)).astype(int32)
A[mymask]=0# non c'è bisogno di fare multiply, controllare nel MAP
mymask

array([   17,    19,    76, ..., 99935, 99975, 99999])

In [9]:
#test per prova 1 e 2
testA=array([[1,100],[2,11],[3,18],[4,130]])
durationstest=testA[:,1]
k=0.6
l=[]
for s in range(4):
    value=durationstest[0]/durationstest[s]
    value=min(value,1/value)
    if(s==0 or value<k):
        l+=[0]
    else:
        l+=[value]
print(l)
firsttest=lil_matrix(l)
for i in range(1,4):
    l=[]
    for s in range(4):
        value=durationstest[i]/durationstest[s]
        value=min(value,1/value)
        if(s==i or value<k):
            l+=[0]
        else:
            l+=[value]
    print(l)
    firsttest=vstack((firsttest,lil_matrix(l)))
firsttest.data

[0, 0, 0, 0.76923076923076927]
[0, 0, 0.61111111111111116, 0]
[0, 0.61111111111111105, 0, 0]
[0.76923076923076916, 0, 0, 0]


array([ 0.76923077,  0.61111111,  0.61111111,  0.76923077])

In [ ]:
# prova 1 non è lungo, è INFINITO
k=0.96 # viene densa circa al 7%
#qualche migliaio di canzoni per riga
l=[]
d=durations[0]
for s in range(100000):
    value=d/durations[s]
    value=min(value,1/value)
    if(s==0 or value<k):
        l+=[0]
    else:
        l+=[value]
#print(l)
j=1
firstRow=lil_matrix(l)
for i in range(1,100000):
    #break
    l=[]
    d=durations[i]
    for s in range(100000):
        value=d/durations[s]
        value=min(value,1/value)
        if(s==i or value<k):
            l+=[0]
        else:
            l+=[value]
    if(j%100==0):
        print(j)
    j+=1
    firstRow=vstack((firstRow,lil_matrix(l)))

In [ ]:
# prova 2 MEMORY ERROR
d=durations[0]
valueArr=d/durations
valueArrInv=1/valueArr
values=array(list(map(lambda x:min(valueArr[x],valueArrInv[x]),range(100000))))
values[where(values<0.95)[0]]=0
values[0]=0
firstRow=csr_matrix(values)
for i in range(1,100000):
    d=durations[i]
    valueArr=d/durations
    valueArrInv=1/valueArr
    values=array(list(map(lambda x:min(valueArr[x],valueArrInv[x]),range(100000))))
    values[where(values<0.95)[0]]=0
    values[i]=0
    firstRow=vstack((firstRow,csr_matrix(values)))
    if (i%1000==0):
        print(i)

In [ ]:
#test per prova 3
k=1.02
for i in range(20):
    d=durations[i]
    dop=where(durations<k*d)[0]
    met=where(durations>d/k)[0]
    ind=intersect1d(dop,met)# indici delle canzoni con durata simile
    print(ind.size)

In [69]:
#prova 3
k=1.5
d=durations[0]
ind=intersect1d(where(durations<k*d)[0],where(durations>d/k)[0])
A=array(zeros(100000))
A[ind]=1
firstRow=csr_matrix(A)
firstRow
for i,d in enumerate(durations[1:]):
    ind=intersect1d(where(durations<k*d)[0],where(durations>d/k)[0])
    A=array(zeros(100000))
    A[ind]=1
    firstRow=vstack((firstRow,csr_matrix(A)))
    if(i%10000==0):
        print(i)

0


KeyboardInterrupt: 

In [70]:
firstRow

<2615x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 150816074 stored elements in Compressed Sparse Row format>

In [71]:
firstRow.size/(2615*100000)

0.5767345086042065